In [ ]:
import json
import boto3
import datetime, time
import random
import requests
import urllib.request
import re
from urllib import parse
from bs4 import BeautifulSoup

def lambda_handler(event, context):
    
    dt_now = datetime.date.today()
    format = '%y.%m.%d'
    ch_dt_now = datetime.datetime.strftime(dt_now, format)
    
    comment_list_result = []
    
    base_url = "https://movie.naver.com/movie/point/af/list.naver?&page={}"
   
    bucket = 'wil-airflow'
    file_name = str(datetime.datetime.now())[:-16]
    
    end_point = 'start'
    for page in range(1,250):
        if end_point == 'end':
            break
        
        url = base_url.format(page)
        res = requests.get(url)
        soup = BeautifulSoup(res.text, "html.parser")
        
        if res.status_code == 200:
            titles = soup.find_all(class_="title")
            dates = soup.find_all("td", class_="num")
            
            for i in range(1,11):
                date = dates[i*2-1].text[-8:]
            
                # 어제 날짜로 바뀌면 종료
                if ch_dt_now != date:
                    end_point = 'end'
                    break
            
                title = titles[i-1].text.split('점 중')[1][:-3]
                missing = title.split('\n')[2]
    
                if missing != '':
                    # 평점
                    score = title.split('\n\n')[0]
                    
                    # 평점 labeling작업 0~6 ->부정(0) 7~10 -> 긍정(1)
                    if int(score) > 6:
                        score = 1
                    else:
                        score = 0
                        
                    # 댓글
                    comment = title.split('\n\n')[1][:-33]
                    # 한글을 제외한 문자 제거
                    commnet_final = re.sub('[^가-힣]', '', comment)
                    
                    # 결측치 제거
                    if commnet_final !='':
                        comment_list_result.append((score, commnet_final))
             
            interval = round(random.uniform(0.2, 1.2), 2)
            time.sleep(interval)
    
    # 중복 값 제거
    comment_list_result = list(set(comment_list_result))
    result = upload_file_s3(bucket, file_name + '.json', comment_list_result)

    if result:
        return {
            'statusCode': 200,
            'body': json.dumps("upload success!")
        }
        
    else:
        return {
            'statusCode': 400,
            'body': json.dumps("upload fail")
        }

# boto3를 활용해 S3 bucket에 저장할 수 있도록 연결해주는 함수
def upload_file_s3(bucket, file_name, file):
    encode_file = bytes(json.dumps(file).encode('UTF-8'))
    s3 = boto3.client('s3')
    try:
        s3.put_object(Bucket=bucket, Key=file_name, Body=encode_file)
        return True
    except:
        return False